<a href="https://colab.research.google.com/github/JulioCezar1111/Projeto30/blob/main/Deep_Learning_RendimentoDoEstudoEAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **`Adaptivity Level`**

# **Importar Bibliotecas**

---



In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn as sk

# **Importar para o dataframe**

---



In [44]:
df = pd.read_csv('/content/drive/MyDrive/students_adaptability_level_online_education.csv')
df.head(1)

,Gender,Age,Education Level,Institution Type,IT Student,Location,Load-shedding,Financial Condition,Internet Type,Network Type,Class Duration,Self Lms,Device,Adaptivity Level
0,Boy,21-25,University,Non Government,No,Yes,Low,Mid,Wifi,4G,3-6,No,Tab,Moderate


# **Tratar Dataframe e separar os dados para as variaveis independentes(X, y)**

---



In [ ]:
df.info()

**Verificar a colunas com valores categorigos**

---



In [36]:
print(df['Gender'].unique()) # Label
print(df['Age'].unique()) # One
print(df['Education Level'].unique()) # One
print(df['Institution Type'].unique()) # Label
print(df['IT Student'].unique()) # Label
print(df['Location'].unique()) # Label
print(df['Load-shedding'].unique()) # Label
print(df['Financial Condition'].unique()) # One
print(df['Internet Type'].unique()) # Label
print(df['Network Type'].unique()) # One
print(df['Class Duration'].unique()) # One
print(df['Self Lms'].unique()) # Label
print(df['Device'].unique()) # One
print(df['Adaptivity Level'].unique()) # label

['Boy' 'Girl']
['21-25' '16-20' '11-15' '26-30' '6-10' '1-5']
['University' 'College' 'School']
['Non Government' 'Government']
['No' 'Yes']
['Yes' 'No']
['Low' 'High']
['Mid' 'Poor' 'Rich']
['Wifi' 'Mobile Data']
['4G' '3G' '2G']
['3-6' '1-3' '0']
['No' 'Yes']
['Tab' 'Mobile' 'Computer']
['Moderate' 'Low' 'High']


**Tratar LabelEncoder**

---



In [45]:
from sklearn.preprocessing import LabelEncoder # converter categoria em numero

cols = ['Gender', 'Institution Type', 'IT Student', 'Location', 'Load-shedding', 'Internet Type', 'Self Lms', 'Adaptivity Level'] #unique pra descobrir quantos rótulos unicos
le = LabelEncoder()
df[cols] = df[cols].apply(le.fit_transform) # fit_transform aprende e transforma

df.head(5)

,Gender,Age,Education Level,Institution Type,IT Student,Location,Load-shedding,Financial Condition,Internet Type,Network Type,Class Duration,Self Lms,Device,Adaptivity Level
0,0,21-25,University,1,0,1,1,Mid,1,4G,3-6,0,Tab,2
1,1,21-25,University,1,0,1,0,Mid,0,4G,1-3,1,Mobile,2
2,1,16-20,College,0,0,1,1,Mid,1,4G,1-3,0,Mobile,2
3,1,11-15,School,1,0,1,1,Mid,0,4G,1-3,0,Mobile,2
4,1,16-20,School,1,0,1,1,Poor,0,3G,0,0,Mobile,1


**Dividir dados para as variaveis independentes**

---



In [ ]:
y = df['Adaptivity Level']
X = df.drop(['Adaptivity Level'], axis = 1)

display(y.head())
display(X.head())

**Tratar OneHotEncoder**

---



In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

coluns = ['Age', 'Education Level', 'Financial Condition', 'Network Type', 'Class Duration', 'Device']

ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), 
                                        coluns)], 
                                       remainder = 'passthrough')

X = ct.fit_transform(X)
X

# **Dividir dataframe de treino e teste**

---



In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 3)

# **Normalizar os dados**

---



In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train

# **Arquitetura da Rede Neural(MLP)**

---



**Bibliotecas**

---



In [59]:
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dropout, BatchNormalization

**Metodo do modelo**

---



In [62]:
from keras.backend import dropout
def create_model(optimizer='Adam'):

    ann = Sequential()
    ann.add(Dense(units = 32, activation='relu', kernel_initializer='he_normal'))
    ann.add(BatchNormalization())

    ann.add(Dense(units = 64, activation='relu', kernel_initializer='he_normal'))
    ann.add(BatchNormalization())

    ann.add(Dense(units = 128, activation='relu', kernel_initializer='he_normal'))
    ann.add(BatchNormalization())

    ann.add(Dense(units = 255, activation='relu', kernel_initializer='he_normal'))
    ann.add(BatchNormalization())

    ann.add(Dense(units = 511, activation='relu', kernel_initializer='he_normal'))
    ann.add(BatchNormalization())
    ann.add(Dropout(0.5))

    ann.add(Dense(units = 3, activation='softmax'))

    ann.compile(optimizer=optimizer, 
                loss='sparse_categorical_crossentropy', 
                metrics=['accuracy'])
    
    return ann

In [63]:
model = KerasClassifier(build_fn=create_model, verbose = 2 )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


**define the grid search parameters part 1**

---



In [64]:
optimizer = ['Adam']
batch_size = [16, 32]
epochs = [30, 50, 70, 80]
param_grid = dict(optimizer=optimizer, batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=None, cv=5)
grid_result = grid.fit(X_train, y_train)

Epoch 1/30
49/49 - 2s - loss: 1.3727 - accuracy: 0.5331 - 2s/epoch - 45ms/step
Epoch 2/30
49/49 - 0s - loss: 0.9101 - accuracy: 0.6796 - 254ms/epoch - 5ms/step
Epoch 3/30
49/49 - 0s - loss: 0.7606 - accuracy: 0.7458 - 248ms/epoch - 5ms/step
Epoch 4/30
49/49 - 0s - loss: 0.7512 - accuracy: 0.7354 - 248ms/epoch - 5ms/step
Epoch 5/30
49/49 - 0s - loss: 0.6128 - accuracy: 0.7678 - 258ms/epoch - 5ms/step
Epoch 6/30
49/49 - 0s - loss: 0.6086 - accuracy: 0.7860 - 269ms/epoch - 5ms/step
Epoch 7/30
49/49 - 0s - loss: 0.5503 - accuracy: 0.7834 - 241ms/epoch - 5ms/step
Epoch 8/30
49/49 - 0s - loss: 0.5941 - accuracy: 0.7782 - 252ms/epoch - 5ms/step
Epoch 9/30
49/49 - 0s - loss: 0.4829 - accuracy: 0.8080 - 253ms/epoch - 5ms/step
Epoch 10/30
49/49 - 0s - loss: 0.5364 - accuracy: 0.7678 - 274ms/epoch - 6ms/step
Epoch 11/30
49/49 - 0s - loss: 0.5070 - accuracy: 0.8106 - 246ms/epoch - 5ms/step
Epoch 12/30
49/49 - 0s - loss: 0.4907 - accuracy: 0.8093 - 268ms/epoch - 5ms/step
Epoch 13/30
49/49 - 0s - lo

**Parametros**

---



In [65]:
batch_size = grid_result.best_params_['batch_size']
epochs = grid_result.best_params_['epochs']
optimizer = grid_result.best_params_['optimizer']

print(batch_size, epochs, optimizer)

32 70 Adam


**Metodo da rede neural com ootimizador**

---



In [66]:
def create_model_Adam(n1 = 32, n2 = 64, n3= 128, n4= 256, n5= 512, learn_rate = 0.01):

    ann = Sequential()
    ann.add(Dense(units = n1, activation='relu', kernel_initializer='he_normal'))
    ann.add(BatchNormalization())

    ann.add(Dense(units = n2, activation='relu', kernel_initializer='he_normal'))
    ann.add(BatchNormalization())

    ann.add(Dense(units = n3, activation='relu', kernel_initializer='he_normal'))
    ann.add(BatchNormalization())

    ann.add(Dense(units = n4, activation='relu', kernel_initializer='he_normal'))
    ann.add(BatchNormalization())

    ann.add(Dense(units = n5, activation='relu', kernel_initializer='he_normal'))
    ann.add(BatchNormalization())
    ann.add(Dropout(0.5))

    ann.add(Dense(units = 3, activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate = learn_rate)

    ann.compile(optimizer=optimizer, 
                loss='sparse_categorical_crossentropy', 
                metrics=['accuracy'])
    
    return ann

In [67]:
model = KerasClassifier(build_fn = create_model_Adam, epochs = epochs, batch_size = batch_size, verbose = 2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


**define the grid search parameters part 2**

---



In [68]:
learn_rate = [0.01, 0.1]
n1 = [32, 64]
n2 = [64, 128]
n3 = [128, 256]
n4 = [256, 512]
n5 = [512, 1024]

param_grid = dict(n1 = n1, n2 = n2, n3 = n3, n4= n4, n5= n5, learn_rate = learn_rate)

grid = GridSearchCV(estimator = model, param_grid = param_grid, n_jobs = None, cv = 5)
grid_result = grid.fit(X_train, y_train)

A saída de streaming foi truncada nas últimas 5000 linhas.
25/25 - 0s - loss: 3.9670 - accuracy: 0.6070 - 197ms/epoch - 8ms/step
Epoch 39/70
25/25 - 0s - loss: 6.6911 - accuracy: 0.5850 - 207ms/epoch - 8ms/step
Epoch 40/70
25/25 - 0s - loss: 2.4315 - accuracy: 0.6446 - 214ms/epoch - 9ms/step
Epoch 41/70
25/25 - 0s - loss: 3.0615 - accuracy: 0.6200 - 200ms/epoch - 8ms/step
Epoch 42/70
25/25 - 0s - loss: 3.1416 - accuracy: 0.5850 - 207ms/epoch - 8ms/step
Epoch 43/70
25/25 - 0s - loss: 2.3587 - accuracy: 0.6615 - 194ms/epoch - 8ms/step
Epoch 44/70
25/25 - 0s - loss: 1.5404 - accuracy: 0.6472 - 208ms/epoch - 8ms/step
Epoch 45/70
25/25 - 0s - loss: 1.4196 - accuracy: 0.6706 - 205ms/epoch - 8ms/step
Epoch 46/70
25/25 - 0s - loss: 1.4126 - accuracy: 0.6680 - 202ms/epoch - 8ms/step
Epoch 47/70
25/25 - 0s - loss: 2.0705 - accuracy: 0.6472 - 196ms/epoch - 8ms/step
Epoch 48/70
25/25 - 0s - loss: 1.6440 - accuracy: 0.6783 - 197ms/epoch - 8ms/step
Epoch 49/70
25/25 - 0s - loss: 1.2270 - accuracy: 0

**Parametros part2**

---



In [69]:
n1 = grid_result.best_params_['n1']
n2 = grid_result.best_params_['n2']
n3 = grid_result.best_params_['n3']
n4 = grid_result.best_params_['n4']
n5 = grid_result.best_params_['n5']
learn_rate = grid_result.best_params_['learn_rate']

print(n1, n2, n3, n4, n5, learn_rate)

32 128 256 512 512 0.01


# **Resultado**

---



In [70]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.877574 using {'learn_rate': 0.01, 'n1': 32, 'n2': 128, 'n3': 256, 'n4': 512, 'n5': 512}
0.853697 (0.036292) with: {'learn_rate': 0.01, 'n1': 32, 'n2': 64, 'n3': 128, 'n4': 256, 'n5': 512}
0.794619 (0.033757) with: {'learn_rate': 0.01, 'n1': 32, 'n2': 64, 'n3': 128, 'n4': 256, 'n5': 1024}
0.841273 (0.017956) with: {'learn_rate': 0.01, 'n1': 32, 'n2': 64, 'n3': 128, 'n4': 512, 'n5': 512}
0.830937 (0.070431) with: {'learn_rate': 0.01, 'n1': 32, 'n2': 64, 'n3': 128, 'n4': 512, 'n5': 1024}
0.840253 (0.040306) with: {'learn_rate': 0.01, 'n1': 32, 'n2': 64, 'n3': 256, 'n4': 256, 'n5': 512}
0.818464 (0.030036) with: {'learn_rate': 0.01, 'n1': 32, 'n2': 64, 'n3': 256, 'n4': 256, 'n5': 1024}
0.859974 (0.019284) with: {'learn_rate': 0.01, 'n1': 32, 'n2': 64, 'n3': 256, 'n4': 512, 'n5': 512}
0.818529 (0.060564) with: {'learn_rate': 0.01, 'n1': 32, 'n2': 64, 'n3': 256, 'n4': 512, 'n5': 1024}
0.859958 (0.024304) with: {'learn_rate': 0.01, 'n1': 32, 'n2': 128, 'n3': 128, 'n4': 256, 'n5': 512}

# **Save best model**

---



In [71]:
best_model = Sequential()
best_model.add(Dense(units = n1, activation='relu', kernel_initializer='he_normal'))
best_model.add(BatchNormalization())

best_model.add(Dense(units = n2, activation='relu', kernel_initializer='he_normal'))
best_model.add(BatchNormalization())

best_model.add(Dense(units = n3, activation='relu', kernel_initializer='he_normal'))
best_model.add(BatchNormalization())

best_model.add(Dense(units = n4, activation='relu', kernel_initializer='he_normal'))
best_model.add(BatchNormalization())

best_model.add(Dense(units = n5, activation='relu', kernel_initializer='he_normal'))
best_model.add(BatchNormalization())
best_model.add(Dropout(0.5))

best_model.add(Dense(units = 3, activation='softmax'))

optimizer = tf.keras.optimizers.Adam(learning_rate = learn_rate)

best_model.compile(optimizer=optimizer, 
            loss='sparse_categorical_crossentropy', 
            metrics=['accuracy'])


best_model.compile(loss='sparse_categorical_crossentropy', optimizer= 'Adam', metrics=['accuracy'])
history = best_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/70
31/31 [==============================] - 2s 10ms/step - loss: 1.3397 - accuracy: 0.5830
Epoch 2/70
31/31 [==============================] - 0s 10ms/step - loss: 0.7402 - accuracy: 0.7376
Epoch 3/70
31/31 [==============================] - 0s 11ms/step - loss: 0.5528 - accuracy: 0.8091
Epoch 4/70
31/31 [==============================] - 0s 11ms/step - loss: 0.5053 - accuracy: 0.8143
Epoch 5/70
31/31 [==============================] - 0s 10ms/step - loss: 0.4883 - accuracy: 0.8475
Epoch 6/70
31/31 [==============================] - 0s 10ms/step - loss: 0.4261 - accuracy: 0.8517
Epoch 7/70
31/31 [==============================] - 0s 10ms/step - loss: 0.3904 - accuracy: 0.8693
Epoch 8/70
31/31 [==============================] - 0s 11ms/step - loss: 0.4215 - accuracy: 0.8579
Epoch 9/70
31/31 [==============================] - 0s 11ms/step - loss: 0.3527 - accuracy: 0.8703
Epoch 10/70
31/31 [==============================] - 0s 12ms/step - loss: 0.3512 - accuracy: 0.8807
Epoch 11/

**Teste do modelo**

---



In [72]:
loss, accuracy = best_model.evaluate(X_test, y_test)
print('Loss: %.2f' % loss)
print('Accuracy: %.2f' % (accuracy*100))

8/8 [==============================] - 0s 4ms/step - loss: 0.2572 - accuracy: 0.9004
Loss: 0.26
Accuracy: 90.04


In [82]:
y_test

999     2
293     2
1035    1
1188    2
752     2
       ..
51      2
31      1
1185    1
175     1
169     1
Name: Adaptivity Level, Length: 241, dtype: int64

# **Save model**

---



In [ ]:
model_json = best_model.to_json()
with open("Modelo01.json", "w") as json_file: # salva o modelo
    json_file.write(model_json)

best_model.save_weights("Modelo01.h5") # salva os pesos da rede
print("Saved model to disk")

# **Load model**

---



In [ ]:
from keras.models import model_from_json
json_file = open('1model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("1model.h5")

print("Loaded model from disk")